In [1]:
#https://github.com/ventolab/CellphoneDB/tree/master/notebooks
##conda activate cpdb
#cellphonedb method statistical_analysis covid_CD16mono_platelet_meta.txt covid_CD16mono_platelet_count.txt --counts-data=gene_name#
#cellphonedb plot dot_plot --rows in/rows.txt --columns in/columns.txt

In [2]:
import pandas as pd
import scanpy as sc

# data after filtering and normalising
adata = sc.read('/home/jovyan/scripts/renal_covid_19/steroid_pipeline/trajectory_with_Rik_model_output/wave2_steroid_2021_covid_GPLVM.h5ad') #inlcude all myeloid annoted



In [3]:
raw= sc.read('/home/jovyan/data/covid_19_renal/renal_covid_19_gex_myeloid_soupX_QC.h5ad')

In [4]:
mdata = raw[raw.obs.index.isin(adata.obs.index)]

In [5]:
mdata

View of AnnData object with n_obs × n_vars = 17256 × 33559

In [6]:
adata

AnnData object with n_obs × n_vars = 17256 × 32913
    obs: 'annot4', 'centre', 'sample_id', 'sample_id_broad', 'sample_date', 'sample_date_yr', 'pool', 'pool_broad', 'haniffa_broad_predLabel', 'orig.ident', 'merged_souporcell_cluster', 'merged_souporcell_status', 'patient_id', 'case_control', 'WHO_severity', 'WHO_temp_severity', 'sex', 'calc_age', 'discharge_date', 'date_positive_swab', 'date_first_symptoms', 'admission_date', 'ethnicity', 'individual_id', 'pseudotime_GPLVM', 'pseudobatch_GPLVM', 'steroid_date', 'days_from_steroid', 'time_from_first_symptoms', 'time_from_positive_swab', 'time_from_infection'
    var: 'GEX'

In [7]:
raw

AnnData object with n_obs × n_vars = 220907 × 33559

In [8]:
mdata.obs['annot4'] = adata.obs['annot4']
mdata.obs['centre'] = adata.obs['centre']
mdata.obs['case_control'] = adata.obs['case_control']
mdata.obs['WHO_severity'] = adata.obs['WHO_severity']
mdata.obs['steroid_date'] = adata.obs['steroid_date']
mdata.obs['days_from_steroid'] = adata.obs['days_from_steroid']

/tmp/ipykernel_3720/3235744017.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  mdata.obs['annot4'] = adata.obs['annot4']
/home/jovyan/my-conda-envs/Py_env2/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [9]:
mdata.obs['annot4'].unique()

['pDC', 'CD14mono_anti_inflammatory', 'RBC', 'CD16mono', 'CD14mono', ..., 'Int.mono', 'CD14mono_activated', 'DC1', 'DC3_IFN', 'ASDC']
Length: 17
Categories (17, object): ['ASDC', 'CD14mono', 'CD14mono_IFN', 'CD14mono_activated', ..., 'Int.mono', 'Platelet', 'RBC', 'pDC']

In [10]:
#sc.pp.subsample(adata, fraction=None, n_obs=100, random_state=0, copy=False)#subsample

In [11]:
mdata.obs['annot4'] = mdata.obs['annot4'].str.replace('CD14mono_activated','CD14mono')
mdata.obs['annot4'] = mdata.obs['annot4'].str.replace('CD14mono_anti_inflammatory','CD14mono_alt_act')

In [12]:
list(mdata.obs['annot4'].unique())

['pDC',
 'CD14mono_alt_act',
 'RBC',
 'CD16mono',
 'CD14mono',
 'CD14mono_IFN',
 'DC2',
 'DC3',
 'Platelet',
 'CD16mono_IFN',
 'HSPC',
 'CD16mono_C1',
 'Int.mono',
 'DC1',
 'DC3_IFN',
 'ASDC']

In [13]:
cell_interested=['CD14mono_alt_act','CD16mono','CD16mono_IFN',
                 'CD16mono_C1','CD14mono','CD14mono_IFN','Int.mono','Platelet']
sub_set=mdata[mdata.obs['annot4'].isin(cell_interested)]


In [14]:
list(sub_set.obs['annot4'].unique())

['CD14mono_alt_act',
 'CD16mono',
 'CD14mono',
 'CD14mono_IFN',
 'Platelet',
 'CD16mono_IFN',
 'CD16mono_C1',
 'Int.mono']

In [15]:
sub_set.obs['centre'].unique()

['Cambridge']
Categories (1, object): ['Cambridge']

In [16]:
sub_set.obs['case_control'].unique()

['POSITIVE', 'RECOVERY']
Categories (2, object): ['POSITIVE', 'RECOVERY']

In [17]:
sub_set = sub_set[sub_set.obs['case_control']=='POSITIVE']

In [18]:
sub_set.obs['case_control'].unique()

['POSITIVE']
Categories (1, object): ['POSITIVE']

In [19]:
sub_set.obs['WHO_severity'].unique()

['moderate', 'severe', 'critical']
Categories (3, object): ['critical', 'moderate', 'severe']

In [20]:
sub_set.obs['days_from_steroid'].unique()

array([  7,  -1,   5,   2,  18,  14,  -5,  11, -10,   9,  -3,   1,  -6,
         6,   4,  -4,   0,  13,  -2,  -8])

In [21]:
sub_set = sub_set[sub_set.obs['days_from_steroid']>0] # for before steroid

In [22]:
sub_set

View of AnnData object with n_obs × n_vars = 8657 × 33559
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'steroid_date', 'days_from_steroid'

In [23]:
sub_set.obs['days_from_steroid'].unique()

array([ 7,  5,  2, 18, 14, 11,  9,  1,  6,  4, 13])

In [24]:
sc.pp.normalize_total(sub_set, target_sum=1e4)

/home/jovyan/.local/lib/python3.8/site-packages/scanpy-1.9.1-py3.8.egg/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [25]:
sc.pp.log1p(sub_set)

In [26]:
sub_set1=sub_set

In [27]:
sub_set1

AnnData object with n_obs × n_vars = 8657 × 33559
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'steroid_date', 'days_from_steroid'
    uns: 'log1p'

In [28]:
# remove vdj and light chain constant genes from highly variable genes, and also the viral reads
import re
for i in sub_set1.var.index:
    if re.search('^IG[HKL][VDJC]|VIRAL', i):
        sub_set1.var.at[i, 'highly_variable'] = False
#sc.pl.highly_variable_genes(sub_set1)

In [29]:
sub_set1 = sub_set1[:, sub_set1.var.highly_variable!=False]
sub_set1

View of AnnData object with n_obs × n_vars = 8657 × 33141
    obs: 'annot4', 'centre', 'case_control', 'WHO_severity', 'steroid_date', 'days_from_steroid'
    var: 'highly_variable'
    uns: 'log1p'

In [30]:
sub_set1.var

,highly_variable
MIR1302-2HG,NaN
FAM138A,NaN
OR4F5,NaN
AL627309.1,NaN
AL627309.3,NaN
...,...
AC233755.2,NaN
AC233755.1,NaN
AC240274.1,NaN
AC213203.1,NaN


In [31]:
df_expr_matrix = sub_set1.X.T ####? should be raw.X???

In [32]:
df_expr_matrix.shape

(33141, 8657)

In [33]:
df_expr_matrix = pd.DataFrame(df_expr_matrix.toarray())

In [34]:
df_expr_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,8647,8648,8649,8650,8651,8652,8653,8654,8655,8656
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df_expr_matrix.columns = sub_set1.obs.index

In [36]:
df_expr_matrix.head()

,UK-CIC10690315+UK-CIC10690507_AAATGCCAGGCACATG-1,UK-CIC10690315+UK-CIC10690507_AACTCTTCACTATCTT-1,UK-CIC10690315+UK-CIC10690507_ACGAGCCCAAAGTCAA-1,UK-CIC10690315+UK-CIC10690507_ACTTTCAGTCTGATTG-1,UK-CIC10690315+UK-CIC10690507_AGCATACGTGTAATGA-1,UK-CIC10690315+UK-CIC10690507_AGCGGTCAGTCTCCTC-1,UK-CIC10690315+UK-CIC10690507_AGCGTATAGATTACCC-1,UK-CIC10690315+UK-CIC10690507_AGCTCCTGTGTAAGTA-1,UK-CIC10690315+UK-CIC10690507_AGGCCGTGTCGTGGCT-1,UK-CIC10690315+UK-CIC10690507_AGTGAGGGTCATACTG-1,...,UK-CIC10690384+UK-CIC10690576_TCAGGATGTCTGGAGA-1,UK-CIC10690384+UK-CIC10690576_TCCACACTCGGAAACG-1,UK-CIC10690384+UK-CIC10690576_TCTTCGGGTATGAAAC-1,UK-CIC10690384+UK-CIC10690576_TGCTGCTAGTTAGCGG-1,UK-CIC10690384+UK-CIC10690576_TGGTTCCAGCGATAGC-1,UK-CIC10690384+UK-CIC10690576_TTAGGCATCCATGAGT-1,UK-CIC10690384+UK-CIC10690576_TTAGTTCGTTCACCTC-1,UK-CIC10690384+UK-CIC10690576_TTCTCAAAGCGCTTAT-1,UK-CIC10690384+UK-CIC10690576_TTGCGTCAGTGGTAAT-1,UK-CIC10690384+UK-CIC10690576_TTTCCTCCACCAGCAC-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
sub_set1.var

,highly_variable
MIR1302-2HG,NaN
FAM138A,NaN
OR4F5,NaN
AL627309.1,NaN
AL627309.3,NaN
...,...
AC233755.2,NaN
AC233755.1,NaN
AC240274.1,NaN
AC213203.1,NaN


In [38]:
#sub_set1.var.set_index('gene_ids',inplace=True)


In [39]:
df_expr_matrix.set_index(sub_set1.var.index, inplace=True) 

In [40]:
df_expr_matrix.head()

,UK-CIC10690315+UK-CIC10690507_AAATGCCAGGCACATG-1,UK-CIC10690315+UK-CIC10690507_AACTCTTCACTATCTT-1,UK-CIC10690315+UK-CIC10690507_ACGAGCCCAAAGTCAA-1,UK-CIC10690315+UK-CIC10690507_ACTTTCAGTCTGATTG-1,UK-CIC10690315+UK-CIC10690507_AGCATACGTGTAATGA-1,UK-CIC10690315+UK-CIC10690507_AGCGGTCAGTCTCCTC-1,UK-CIC10690315+UK-CIC10690507_AGCGTATAGATTACCC-1,UK-CIC10690315+UK-CIC10690507_AGCTCCTGTGTAAGTA-1,UK-CIC10690315+UK-CIC10690507_AGGCCGTGTCGTGGCT-1,UK-CIC10690315+UK-CIC10690507_AGTGAGGGTCATACTG-1,...,UK-CIC10690384+UK-CIC10690576_TCAGGATGTCTGGAGA-1,UK-CIC10690384+UK-CIC10690576_TCCACACTCGGAAACG-1,UK-CIC10690384+UK-CIC10690576_TCTTCGGGTATGAAAC-1,UK-CIC10690384+UK-CIC10690576_TGCTGCTAGTTAGCGG-1,UK-CIC10690384+UK-CIC10690576_TGGTTCCAGCGATAGC-1,UK-CIC10690384+UK-CIC10690576_TTAGGCATCCATGAGT-1,UK-CIC10690384+UK-CIC10690576_TTAGTTCGTTCACCTC-1,UK-CIC10690384+UK-CIC10690576_TTCTCAAAGCGCTTAT-1,UK-CIC10690384+UK-CIC10690576_TTGCGTCAGTGGTAAT-1,UK-CIC10690384+UK-CIC10690576_TTTCCTCCACCAGCAC-1
MIR1302-2HG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AL627309.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df_expr_matrix.shape

(33141, 8657)

In [42]:
#df_expr_matrix=df_expr_matrix.loc[~(df_expr_matrix==0).all(axis=1)]

In [43]:
df_expr_matrix.shape

(33141, 8657)

In [44]:
df_expr_matrix.to_csv('/home/jovyan/mounting_point/cellphoneDB_output/allmono_platelet_wave2_pos_steroid_count_after.txt',sep='\t')

In [45]:
sub_set1.obs

,annot4,centre,case_control,WHO_severity,steroid_date,days_from_steroid
UK-CIC10690315+UK-CIC10690507_AAATGCCAGGCACATG-1,CD14mono_alt_act,Cambridge,POSITIVE,moderate,28/01/2021,7
UK-CIC10690315+UK-CIC10690507_AACTCTTCACTATCTT-1,CD16mono,Cambridge,POSITIVE,moderate,28/01/2021,7
UK-CIC10690315+UK-CIC10690507_ACGAGCCCAAAGTCAA-1,CD14mono,Cambridge,POSITIVE,moderate,28/01/2021,7
UK-CIC10690315+UK-CIC10690507_ACTTTCAGTCTGATTG-1,CD14mono,Cambridge,POSITIVE,moderate,28/01/2021,7
UK-CIC10690315+UK-CIC10690507_AGCATACGTGTAATGA-1,CD14mono_alt_act,Cambridge,POSITIVE,moderate,28/01/2021,7
...,...,...,...,...,...,...
UK-CIC10690384+UK-CIC10690576_TTAGGCATCCATGAGT-1,CD14mono,Cambridge,POSITIVE,severe,07/01/2021,11
UK-CIC10690384+UK-CIC10690576_TTAGTTCGTTCACCTC-1,CD14mono,Cambridge,POSITIVE,severe,07/01/2021,11
UK-CIC10690384+UK-CIC10690576_TTCTCAAAGCGCTTAT-1,CD14mono_IFN,Cambridge,POSITIVE,severe,07/01/2021,11
UK-CIC10690384+UK-CIC10690576_TTGCGTCAGTGGTAAT-1,CD14mono,Cambridge,POSITIVE,severe,07/01/2021,11


In [46]:
sub_set1.obs.reset_index(inplace=True)

In [47]:
sub_set1.obs.head()

,index,annot4,centre,case_control,WHO_severity,steroid_date,days_from_steroid
0,UK-CIC10690315+UK-CIC10690507_AAATGCCAGGCACATG-1,CD14mono_alt_act,Cambridge,POSITIVE,moderate,28/01/2021,7
1,UK-CIC10690315+UK-CIC10690507_AACTCTTCACTATCTT-1,CD16mono,Cambridge,POSITIVE,moderate,28/01/2021,7
2,UK-CIC10690315+UK-CIC10690507_ACGAGCCCAAAGTCAA-1,CD14mono,Cambridge,POSITIVE,moderate,28/01/2021,7
3,UK-CIC10690315+UK-CIC10690507_ACTTTCAGTCTGATTG-1,CD14mono,Cambridge,POSITIVE,moderate,28/01/2021,7
4,UK-CIC10690315+UK-CIC10690507_AGCATACGTGTAATGA-1,CD14mono_alt_act,Cambridge,POSITIVE,moderate,28/01/2021,7


In [48]:
df_meta = pd.DataFrame(data={'Cell': list(sub_set1.obs['index']), 
                             'cell_type': list(sub_set1.obs['annot4'])})

In [49]:
df_meta.head()

,Cell,cell_type
0,UK-CIC10690315+UK-CIC10690507_AAATGCCAGGCACATG-1,CD14mono_alt_act
1,UK-CIC10690315+UK-CIC10690507_AACTCTTCACTATCTT-1,CD16mono
2,UK-CIC10690315+UK-CIC10690507_ACGAGCCCAAAGTCAA-1,CD14mono
3,UK-CIC10690315+UK-CIC10690507_ACTTTCAGTCTGATTG-1,CD14mono
4,UK-CIC10690315+UK-CIC10690507_AGCATACGTGTAATGA-1,CD14mono_alt_act


In [50]:
df_meta.set_index('Cell',inplace=True)
df_meta.to_csv('/home/jovyan/mounting_point/cellphoneDB_output/allmono_platelet_wave2_pos_steroid_meta_after.txt', sep='\t')